In [1]:
import pandas as pd
import sqlite3

In [2]:
connection = sqlite3.connect("../data/checking-logs.sqlite")

In [3]:
cursor = connection.cursor()

In [ ]:
cursor.execute("""
CREATE TABLE datamart AS
SELECT 
    checker.uid AS uid,
    checker.labname AS labname,
    MIN(checker.timestamp) AS first_commit_ts,
    MIN(pageviews.datetime) AS first_view_ts
FROM checker
LEFT JOIN pageviews ON checker.uid = pageviews.uid
WHERE checker.status = 'ready'
    AND numTrials = 1
    AND labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
    AND checker.uid LIKE 'user_%'
GROUP BY checker.uid, checker.labname;
""")

In [ ]:
pd.io.sql.read_sql("SELECT * FROM datamart LIMIT 10;", connection)

In [5]:
query = "SELECT * FROM datamart;"
datamart_df = pd.read_sql_query(query, connection)

datamart_df['first_commit_ts'] = pd.to_datetime(datamart_df['first_commit_ts'])
datamart_df['first_view_ts'] = pd.to_datetime(datamart_df['first_view_ts'])

datamart_df.head()

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136


In [6]:
test_df = datamart_df[datamart_df['first_view_ts'].notna()]
control_df = datamart_df[datamart_df['first_view_ts'].isna()]

In [7]:
avg_first_view_ts = test_df['first_view_ts'].mean()
control_df.loc[:, 'first_view_ts'] = avg_first_view_ts


In [8]:
test_df.to_sql("test", connection, if_exists="replace", index=False)
control_df.to_sql("control", connection, if_exists="replace", index=False)

81

In [9]:
connection.close()